In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import text, sequence

In [2]:
df_tr = pd.read_csv("../input/train.csv")
df_te = pd.read_csv("../input/test.csv")

In [9]:
list_sentences_train = df_tr["comment_text"].values
list_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y = df_tr[list_classes].values

In [11]:
list_sentences_test = df_te["comment_text"].values

In [14]:
maxlen = 70
max_features = 95000
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen = maxlen)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen = maxlen)

In [27]:
print(X_train.shape, X_test.shape)

(159571, 70) (153164, 70)


In [33]:
nb_words = len(tokenizer.word_index)
from keras.models import Model
from keras.layers import Input, Embedding, CuDNNGRU, GlobalMaxPooling1D, Dense
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, 100)(inp)
    x = CuDNNGRU(40, return_sequences=True)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=['accuracy'])
    return model

In [34]:
model = get_model()

In [36]:
model.fit(X_train, y, batch_size=40, epochs=1)

Epoch 1/1
159571/159571 [==============================] - 177s 1ms/step - loss: 0.0716 - acc: 0.9776


In [37]:
y_test = model.predict(X_test)

In [38]:
y_test

array([[9.4808120e-01, 2.6936373e-01, 9.3746424e-01, 8.5131012e-02,
        7.7313054e-01, 1.4870942e-01],
       [2.4146698e-03, 8.7551154e-05, 3.4507681e-04, 2.0295579e-04,
        3.5453992e-04, 3.4739281e-04],
       [5.1158429e-03, 1.0759173e-04, 7.2331977e-04, 2.5846338e-04,
        5.8319204e-04, 4.3033244e-04],
       ...,
       [2.5164583e-03, 4.6306861e-05, 5.0274271e-04, 7.8445970e-05,
        3.6447088e-04, 2.1146030e-04],
       [2.0109085e-02, 5.7713164e-04, 6.2939147e-03, 7.0753240e-04,
        4.9976469e-03, 1.5933007e-03],
       [8.9760441e-01, 1.5283803e-02, 5.6369883e-01, 9.3293926e-03,
        3.2159272e-01, 2.5219105e-02]], dtype=float32)